# IMPORT MODULES

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
# from torchmetrics.regression import SpearmanCorrCoef, KendallRankCorrCoef
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
import copy

from __future__ import print_function
import argparse
import torch.utils.data
from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from sklearn.utils import shuffle

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
# import dill
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from scipy.stats import gaussian_kde, spearmanr, pearsonr
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA, KernelPCA
import seaborn as sns
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 3, 3
sns.set(rc={'figure.figsize':(3,3)})
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error,median_absolute_error, balanced_accuracy_score
import os
# import chime
# chime.theme('mario')
np.random.seed(42)
from tqdm import trange
from IPython.display import Image
from scipy.special import kl_div
import h2o
from h2o.automl import H2OAutoML
import prince

In [ ]:
from sdv.single_table import TVAESynthesizer, CTGANSynthesizer, CopulaGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [ ]:
# pip install ImbalancedLearningRegression-master.zip
from ImbalancedLearningRegression import smote, ro, gn, adasyn

In [ ]:
rep = 'C:/Users/samgo/OneDrive/Perso/Thèse/Travaux/VAKIR/XP4/'

# DATA

In [ ]:
data = pd.read_csv("C:/Users/samgo/OneDrive/Perso/Thèse/Travaux/DataSets-IR/boston.csv")
context="supervised"

In [ ]:
data.info()

In [ ]:
lab_Y = "HousValue"
data.dropna(axis=0, inplace=True)

In [ ]:
data

In [ ]:
data.dropna(axis=0, inplace=True)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
sns.histplot(data[lab_Y])

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.pairplot(data,diag_kind='kde')
plt.show()

In [ ]:
# X_df = pd.read_csv(rep+"/X_df.csv")
# data = X_df

In [ ]:
kdem = gaussian_kde(np.array(data.T),bw_method="scott")
pd.DataFrame(kdem.covariance)

In [ ]:
def transformer(inputs, mode_quanti="MinMax", mode_quali="OHE"):
    quanti = inputs.select_dtypes(include=['number'])
    quali = inputs.select_dtypes(include=['object', 'category'])
    name_quanti = quanti.columns
    if mode_quanti=="MinMax" :
      scaler = MinMaxScaler()
      quanti_t = pd.DataFrame(scaler.fit_transform(quanti))
    else:
      scaler = StandardScaler()
      quanti_t = pd.DataFrame(scaler.fit_transform(quanti))
    quanti_t.columns = name_quanti
    if quali.shape[1]==0:
        return quanti_t
    else:
        quali_t = pd.get_dummies(quali)
        if mode_quali=="AFDM" :
          quali_t=quali_t/np.sqrt(quali_t.mean())
        return pd.concat([quanti_t, quali_t], axis=1)

In [ ]:
def inv_transformer(inputs, outputs, mode_quanti="MinMax", mode_quali="OHE"):
  quanti = inputs.select_dtypes(include=['number'])
  quali = inputs.select_dtypes(include=['object', 'category'])
  name_quanti = quanti.columns
  if quanti.shape[1] > 0 :
    if mode_quanti=="MinMax" :
      scaler = MinMaxScaler()
      scaler.fit(quanti)
      res_quanti =  pd.DataFrame(scaler.inverse_transform(outputs[name_quanti]))
    elif mode_quanti=="SC" :
      scaler = StandardScaler()
      scaler.fit(quanti)
      res_quanti =  pd.DataFrame(scaler.inverse_transform(outputs[name_quanti]))
    else:
      res_quanti =  pd.DataFrame(outputs[name_quanti])
    res_quanti.columns = name_quanti
    res_quanti.index = outputs.index
  if quali.shape[1] > 0 :
    quali_t = pd.get_dummies(quali)
    name_quali_t = quali_t.columns
    res_quali = outputs.copy()
    res_quali.drop(name_quanti, axis=1, inplace=True)
    res_quali.index = quali_t.index
    res_quali = np.round(res_quali,0)
    if quanti.shape[1] > 0 :
      return pd.concat([res_quanti, res_quali], axis=1)
    else:
      return res_quali
  else:
    return res_quanti

In [ ]:
colY = np.where(data.columns==lab_Y)[0][0] ; colY

## Pondération IR 

In [ ]:
def IR_weighting(Y, plot=False, alpha=1/2):
    if alpha is None:
        w = Y*0+1
    else:
        w= 1/gaussian_kde(Y).evaluate(Y)**(alpha)
    w=w/sum(w)
    if plot==True:
        sns.set(rc={'figure.figsize':(3,3)})
        sns.histplot(Y)
        plt.show()
        sns.scatterplot(x=Y,y=w)
    return w

In [ ]:
alfa=1

In [ ]:
w_Y=IR_weighting(data[lab_Y], plot=True, alpha=alfa)

## Train-Test Split

In [ ]:
test_size=0.4
train_size=0.6

In [ ]:
def trainTest(data, test_size=0.3,np_seed=None,w=None, train_size=None):
    if np_seed is None:
        np.random.seed()
    else:
        np.random.seed(np_seed)
    n = data.shape[0]
    if w is None:w=np.repeat(1,n)/n
    n_test=round(test_size*n)
    id_test = np.random.choice(n, size=n_test, p=w , replace=False)
    X_test = data.iloc[id_test,:]
    X_train = data.drop(index=id_test)
    if train_size is not None:X_train=X_train.sample(round(train_size*n))
    
    return{'X_train':X_train,'X_test':X_test}

In [ ]:
seed_sample = np.random.randint(1000);seed_sample

Tirage uniforme

In [ ]:
split = trainTest(data,test_size,seed_sample,w=w_Y,train_size=train_size)
X_train = split['X_train']
X_test = split['X_test']
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train.describe()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
train_df = pd.DataFrame(X_train[[lab_Y]])
train_df['sample']='train'
test_df = pd.DataFrame(X_test[[lab_Y]])
test_df['sample']='test'
sns.histplot(data=pd.concat([train_df,test_df],axis=0),x=lab_Y,hue='sample',kde=True, stat="density")

In [ ]:
print(X_train[[lab_Y]].describe())
sns.histplot(X_train[[lab_Y]])

In [ ]:
print(X_test[[lab_Y]].describe())
sns.histplot(X_test[[lab_Y]])

In [ ]:
Xy_train=X_train.copy()
if context=="supervised":
    y_train =  X_train[[lab_Y]]
    X_train.drop(data.columns[colY], axis=1, inplace=True)
    w_Y=IR_weighting(y_train[lab_Y], alpha=alfa)
    sns.scatterplot(x=y_train[lab_Y],y=w_Y)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.pairplot(data,diag_kind='kde')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(np.round(data.corr(method="pearson"),1),annot=True, cmap="Reds")

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(np.round(data.corr(method="spearman"),1),annot=True, cmap="RdYlGn")

In [ ]:
# X_df.to_csv(rep+'X_df.csv', index=False)

# PARAMETRES

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

In [ ]:
encDim_S = 5
encDim_M = 10
encDim_L = 15

hp_fae_w_corr = None
hp_fae_sparsity_factor = None
hp_fae_sparsity_target=0.05
hp_fae_sparsity_mode=None
hp_fae_wNorm=False
hp_fae_mirror=False
hp_fae_encDim=1
hp_fae_epochs=1000
hp_fae_batch_size=64
hp_fae_lr=10e-4
#, BN_thr=0.05
hp_fae_lossfunc="wMSE"
hp_fae_dimHL=5
hp_fae_w_Y = None
hp_dimLat = 10 
hp_betaKLD=1e-6 
hp_betaX=1 
hp_betaY=10

hp_power=1
hp_penalRank=1

plotModel=False

# ---------------------------- INDUSTRIALISATION

## Model Definition

### VanillaAEy

In [ ]:
class AEy(nn.Module):
    def __init__(self, p,dimHL=hp_fae_dimHL, power = hp_power,penal_rank=hp_penalRank, dimLat=hp_dimLat,connect=True, betaKLD=None):
        super().__init__()
        self.p = p
        self.dimLat = dimLat
        q = int(p/10)+1
        self.encoders = nn.Sequential(
                    nn.Linear(p, 2*p),
                    nn.Tanh(),
                    nn.Linear(2*p, p-1*q),
                    nn.Tanh(),
                    nn.Linear(p-1*q, p-2*q),
                    nn.Tanh(),
                    nn.Linear(p-2*q, p-3*q))

        self.decoders = nn.Sequential(
                    nn.Linear(p-3*q, p-2*q),
                    nn.Tanh(),
                    nn.Linear(p-2*q, p-1*q),
                    nn.Tanh(),
                    nn.Linear(p-1*q, 2*p),
                    nn.Tanh(),
                    nn.Linear(2*p, p))
        

    def encode(self, x) :
        encoded=self.encoders(x)
        return encoded
    
    def decode(self, z):
        decoded = self.decoders(z)
        return decoded[:,:-1],decoded[:,[-1]]
    
    
    def forward(self, x,y):
        encoded = self.encode(torch.cat((x,y),dim=1))  
        decoded_x, decode_y = self.decode(encoded)
        return decoded_x, decode_y, encoded, encoded*0, encoded*0 

    


### VanillaVAEy

In [ ]:
class VAEy(nn.Module):
    def __init__(self, p,dimHL=hp_fae_dimHL,sparsity_factor=None, power = hp_power,penal_rank=hp_penalRank, dimLat=hp_dimLat,connect=True, betaKLD=None):
        super().__init__()
        self.p = p
        self.dimLat = p-3
        self.betaKLD = betaKLD
        q = int(p/10)+1
        self.fc1 = nn.Linear(p+1, 2*p+1)
        self.fc2 = nn.Linear(2*p+1, p-1*q) 
        self.fc3 = nn.Linear(p-1*q, p-2*q) 
        self.fc41 = nn.Linear(p-2*q, p-3*q)
        self.fc42 = nn.Linear(p-2*q,p-3*q) 

        self.fc5 = nn.Linear(p-3*q, p-2*q)
        self.fc6 = nn.Linear(p-2*q, p-1*q) 
        self.fc7 = nn.Linear(p-1*q, 2*p+1) 
        self.fc81 = nn.Linear(2*p+1, p)
        self.fc82 = nn.Linear(2*p+1, 1) 
               
    def encode(self, x, a) :
        h1 = F.tanh(self.fc1(torch.cat([x.view(-1, x.shape[1]),a.view(-1, 1)],1)))  # on utilise une simple ReLu pour la premiere couche de 400 neurones et ID pour la deuxieme couche pour u et std
        h2 = F.tanh(self.fc2(h1))
        h3 = F.tanh(self.fc3(h2))
        mu = self.fc41(h3)
        logvar = self.fc42(h3)
        return mu, logvar
              

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)  # racine de la variance ==> std
        eps = torch.randn_like(std)   # une loi normal entre 0 et 1 avec une taille de std soit de 20 !!
        return mu + eps*std  # z = u + sigm * eps
  
    def decode(self, z):
        h5 = F.tanh(self.fc5(z))
        h6 = F.tanh(self.fc6(h5))
        h7 = F.tanh(self.fc7(h6))
        decoded_x = self.fc81(h7)
        decoded_y = self.fc82(h7)
        return decoded_x, decoded_y
    
    
    def forward(self, x, a):
        mu,logvar = self.encode(x.view(-1, x.shape[1]), a) 
        encoded = self.reparameterize(mu, logvar)
        decoded_x, decode_y = self.decode(encoded)
        return decoded_x, decode_y, encoded, mu, logvar 
        


### Loss Functions

In [ ]:
def balanced_mse_loss(recon_y, y, w):
    # print("torch.nn.functional.mse_loss(recon_y, y, reduction='sum')",torch.nn.functional.mse_loss(recon_y, y, reduction='sum'))
    # print("torch.sum(w*(recon_y-y).T**2",torch.sum(w*(recon_y-y).T**2))
    return torch.sum(w*(recon_y-y).T**2)

In [ ]:
def loss_function_VAE(recon_x, z, recon_y, y, x, mu, logvar, epoch,i, betaX = 1,betaY = 0,  betaKLD = 0.5, w_Y=None):
    BCE_X = torch.nn.functional.mse_loss(recon_x, x, reduction='sum')
    if w_Y is None:
        BCE_Y = torch.nn.functional.mse_loss(recon_y, y, reduction='sum')
    else:
        BCE_Y = balanced_mse_loss(recon_y, y, w=w_Y)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # ici c'est la KL pour une loi normal 0 & 1 uniquement
    return betaX*BCE_X + betaKLD*KLD + betaY*BCE_Y , betaY*BCE_Y , betaX*BCE_X  , betaKLD*KLD  #+ mmd  # ici on additionne tout simplement les deux pour la loss function avec BCE pour retrouver X et KLD pour ne pas s'éloigner d'une loi normale


## Model Training

### AE-X & VAE-X

In [ ]:
def train_VAE(X_train,y_train, seed=seed_sample, lr=hp_fae_lr, batch_size=hp_fae_batch_size, epochs=hp_fae_epochs, dimHL=hp_fae_dimHL,dimLat=hp_dimLat,stacked=True, 
              power=hp_power,penal_rank=hp_penalRank, betaX = hp_betaX,betaY = 0,  betaKLD=hp_betaKLD, connect=False,type_model="VAE"):
        
        print("betaX",betaX)
        print("betaKLD",betaKLD)
        X_train = transformer(X_train)
        X_train = np.array(X_train)
        y_train = transformer(y_train)
        y_train = np.array(y_train)
        
        inputsX=torch.FloatTensor(X_train).to(device)
        inputsY=torch.FloatTensor(y_train).to(device)
        p = X_train.shape[1]
        if type_model=="VAE":
            print("type model : VAE")
            model = VAE(p,dimHL=dimHL, power = power,penal_rank=penal_rank, dimLat=dimLat, connect=connect).to(device)
        elif type_model=="AE":
            print("type model : AE")
            model = AE(p,dimHL=dimHL, power = power,penal_rank=penal_rank, dimLat=dimLat, connect=connect).to(device)

        
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001)
        batch_idx=0
        # log_interval=50
        batch_no = len(X_train) // batch_size

        decod=[]
        losses=[]
        losses_VAE=[]
        losses_X=[]
        losses_Y=[]
        losses_KL=[]
        
        for epoch in trange(epochs, desc="Proving P=NP", unit="carrots"):
          x_train,  ytrain = shuffle(X_train, np.expand_dims(y_train,axis = 1))
            # Mini batch learning
          for i in range(batch_no):
            start = i * batch_size
            end = start + batch_size
            x_var = Variable(torch.FloatTensor(x_train[start:end])).to(device)
            y_var = Variable(torch.FloatTensor(ytrain[start:end])).to(device)

            # model.train()
            train_loss = 0
            optimizer.zero_grad()
            decoded, encoded, mu, logvar = model(x_var)
            loss_VAE, loss_Y, loss_X, loss_KL = loss_function_VAE(decoded, encoded,y_var, y_var, x_var, mu, logvar,epoch,i, betaX = betaX,betaY = betaY,  betaKLD = betaKLD)
            if type_model=="AE":
                loss = loss_X
            else:
                loss = loss_VAE
            loss.backward()
            train_loss += loss.item()
            losses.append(loss.cpu().detach().numpy()+0)
            losses_VAE.append(loss_VAE.cpu().detach().numpy()+0)
            losses_X.append(loss_X.cpu().detach().numpy()+0)
            # losses_Y.append(loss_Y.cpu().detach().numpy()+0)
            losses_KL.append(loss_KL.cpu().detach().numpy()+0)
            optimizer.step()
          decoded, encoded, mu, logvar = model(inputsX)
          if type_model=="AE":
                mu=encoded
        return {'model':model, "decoded":decoded, "encoded":encoded, "mu":mu, "logvar":logvar,
                'losses':losses, 'losses_VAE':losses_VAE, 'losses_X':losses_X, 'losses_KL':losses_KL}


### AE-Xy & VAE-Xy

In [ ]:
def train_VAEy(X_train,y_train, seed=seed_sample, lr=hp_fae_lr, batch_size=hp_fae_batch_size, epochs=hp_fae_epochs, dimHL=hp_fae_dimHL,dimLat=hp_dimLat,stacked=True, 
              power=hp_power,penal_rank=hp_penalRank, betaX = hp_betaX,betaY = hp_betaY,  betaKLD=hp_betaKLD, connect=False,type_model="VAE", w_Y=None):
        
        torch.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        
        print("betaX",betaX)
        print("betaKLD",betaKLD)
        print("betaY",betaY)
        X_train = transformer(X_train)
        y_train = transformer(y_train)
        inputsX=torch.FloatTensor(X_train.to_numpy()).to(device)
        inputsY=torch.FloatTensor(y_train.to_numpy()).to(device)
        if w_Y is not None:
            w_Y=torch.FloatTensor(w_Y)*y_train.shape[0]
        else:
            w_Y=torch.FloatTensor(np.repeat(1,y_train.shape[0]))
        p = X_train.shape[1]
        if type_model=="VAE":
            print("type model : VAEy")
            model = VAEy(p,dimHL=dimHL, power = power,penal_rank=penal_rank, dimLat=dimLat, connect=connect).to(device)
        elif type_model=="AE":
            print("type model : AEy")
            model = AEy(p+1,dimHL=dimHL, power = power,penal_rank=penal_rank, dimLat=dimLat, connect=connect).to(device)
            
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001)
        batch_idx=0
        # log_interval=50
        batch_no = len(X_train) // batch_size

        decod=[]
        losses=[]
        losses_VAE=[]
        losses_X=[]
        losses_Y=[]
        losses_KL=[]
        
        for epoch in trange(epochs, desc="Proving P=NP", unit="carrots"):
          # x_train,  ytrain = shuffle(X_train, np.expand_dims(y_train,axis = 1))
          xtrain, ytrain, wtrain = shuffle(inputsX, inputsY, w_Y)
          # Mini batch learning
          for i in range(batch_no):
            start = i * batch_size
            end = start + batch_size
            x_var = Variable(xtrain[start:end]).to(device)
            y_var = Variable(ytrain[start:end]).to(device)
            w_var = Variable(wtrain[start:end]).to(device)
            # plt.scatter(y_var.cpu().detach().numpy(),w_var.cpu().detach().numpy())
            # plt.show()

            # model.train()
            train_loss = 0
            optimizer.zero_grad()
            decoded_x, decoded_y, encoded, mu, logvar = model(x_var, y_var)
            loss_VAE, loss_Y, loss_X, loss_KL = loss_function_VAE(decoded_x, encoded, decoded_y, y_var, x_var, mu, logvar,epoch,i, betaX = betaX,betaY = betaY,  
                                                                  betaKLD = betaKLD, w_Y=w_var)
            loss = loss_VAE
            loss.backward()
            train_loss += loss.item()
            losses.append(loss.cpu().detach().numpy()+0)
            losses_VAE.append(loss_VAE.cpu().detach().numpy()+0)
            losses_X.append(loss_X.cpu().detach().numpy()+0)
            losses_Y.append(loss_Y.cpu().detach().numpy()+0)
            losses_KL.append(loss_KL.cpu().detach().numpy()+0)
            optimizer.step()
          decoded_x, decoded_y, encoded, mu, logvar = model(inputsX, inputsY)
        if type_model=="AE":
                mu=encoded
        return {'model':model, "decoded_x":decoded_x, "decoded_y":decoded_y, "encoded":encoded, "mu":mu, "logvar":logvar,
                'losses':losses, 'losses_VAE':losses_VAE, 'losses_X':losses_X, 'losses_KL':losses_KL, 'losses_Y':losses_Y}


In [ ]:
def KL_data(r,s):
    P = gaussian_kde(r).evaluate(s)
    Q = gaussian_kde(s).evaluate(s)
    return kl_div(P,Q).sum()

In [ ]:
def KL_dataset(Synth):
    res = 0
    for col in Synth.columns:
        if col != 'Type':
            res = res + KL_data(Synth[Synth.Type=='Real'][col],Synth[Synth.Type=='Recons'][col])
    return res


------------------------------------

------------------------------

## Automatisation

In [ ]:
def PCA_NL(X_train, y_train=None,betaKLD=hp_betaKLD, stacked=False, epochs=hp_fae_epochs,seed=np.random.randint(1000), connect=False, type_model="VAE"):
    if y_train is None: np.repaat(1, train.shape[O])
    # Linéarisation
    res_LSFVAE = train_VAE(X_train=X_train,y_train=y_train, seed=seed_sample, epochs=epochs, stacked=stacked, betaKLD=betaKLD, connect=connect, type_model=type_model)
    
    # Sorties
    decoded = res_LSFVAE['decoded']
    encoded = pd.DataFrame(res_LSFVAE['encoded'].cpu().detach().numpy())
    
    return {'res_LSFVAE':res_LSFVAE, 'encoded':encoded}

In [ ]:
def PCA_NL_Analyze(res, X_train, plot_Model=False, MC_ref=None):
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    train=X_train.copy()
    decoded = pd.DataFrame(res['res_LSFVAE']['decoded'].cpu().detach().numpy())
    encoded = res['encoded']
    decoded.columns=X_train.columns
    RMSE_X = np.sqrt(np.mean((decoded-transformer(train))**2))
    decoded = inv_transformer(X_train, decoded)
    decoded['Type'] = 'Recons'
    train['Type'] = 'Real'
    reconstruct  = pd.concat([train,decoded])
    KL_R = 0#KL_dataset(reconstruct)
    if plot_Model:
        losses=res['res_LSFVAE']['losses']
        losses_VAE=res['res_LSFVAE']['losses_VAE']
        losses_X=res['res_LSFVAE']['losses_X']
        losses_KL=res['res_LSFVAE']['losses_KL']
        plt.clf()
        m=int(len(losses)/50)
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.lineplot(pd.DataFrame({"losses":losses,"losses_VAE":losses_VAE,"losses_X":losses_X,"losses_KL":losses_KL}))
        plt.suptitle("Losses Analysis", y=1.02)
        plt.show()
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.lineplot(pd.DataFrame({"losses":losses,"losses_VAE":losses_VAE,"losses_X":losses_X,"losses_KL":losses_KL}).iloc[m:,:])
        plt.suptitle("Losses Analysis-Focus", y=1.02)
        plt.show()
        print("losses_min:",np.min(losses))
        print("RMSE_X:",RMSE_X)
        print("MSE_X:",RMSE_X**2)
    if  plot_Model:
        sns.pairplot(encoded,diag_kind='kde')
        plt.show()
    # if plot_Model:
    #     sns.set(rc={'figure.figsize':(11.7,8.27)})
    #     sns.heatmap(np.round(pd.DataFrame(encoded.corr(method="spearman")),1),annot=True)
    #     sns.heatmap(np.round(pd.DataFrame(encoded.corr(method="pearson")),1),annot=True)
    #     plt.show()
    # if MC_ref is not None:
    #     MC_dif = (np.abs(MC_ref - np.abs(pd.DataFrame(encoded.corr(method="spearman")))).sum().sum())/2
    #     MC_filtre = MC_ref - np.eye(MC_ref.shape[1])
    #     MC_corr=np.abs(np.abs(pd.DataFrame(encoded.corr(method="spearman"))) * MC_filtre - MC_filtre).sum().sum()/2 
    if plot_Model:  
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.pairplot(decoded, diag_kind='kde')
        sns.pairplot(reconstruct, diag_kind='kde', hue='Type')
        
    if MC_ref is not None:
        return {'RMSE_X':RMSE_X,'MSE_X':RMSE_X**2, 'MC_dif':MC_dif,'MC_corr':MC_corr,"KL_R":KL_R}
    else:
        return {'RMSE_X':RMSE_X, 'MSE_X':RMSE_X**2, "KL_R":KL_R}

In [ ]:
def PCA_NL_Analyze_y(res, X_train, y_train, plot_Model=False, MC_ref=None, w_Y=None):
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    train=X_train.copy()
    ytrain = pd.DataFrame(y_train.copy())
    ytrain.columns = [lab_Y]
    decoded_x = pd.DataFrame(res['decoded_x'].cpu().detach().numpy())
    decoded_y = pd.DataFrame(res['decoded_y'].cpu().detach().numpy())
    encoded = pd.DataFrame(res['encoded'].cpu().detach())
    decoded_x.columns=X_train.columns
    decoded_y.columns=ytrain.columns
    RMSE_X = np.sqrt(np.mean((decoded_x-transformer(train))**2))
    RMSE_y = np.sqrt(np.mean((decoded_y-transformer(ytrain))**2))
    wRMSE_y = np.sqrt(np.mean(((decoded_y-transformer(y_train))**2)[lab_Y]*w_Y))
    decoded_x = inv_transformer(X_train, decoded_x)
    decoded_x['Type'] = 'Recons'
    train['Type'] = 'Real'
    reconstruct_x  = pd.concat([train,decoded_x])
    
    decoded_y = inv_transformer(ytrain, decoded_y)
    decoded_y['Type'] = 'Recons'
    ytrain['Type'] = 'Real'
    reconstruct_y  = pd.concat([ytrain,decoded_y])
    
    KL_R = 0#KL_dataset(reconstruct)
    if plot_Model:
        losses=res['losses']
        losses_VAE=res['losses_VAE']
        losses_X=res['losses_X']
        losses_Y=res['losses_Y']
        losses_KL=res['losses_KL']
        plt.clf()
        m=int(len(losses)/50)
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.lineplot(pd.DataFrame({"losses":losses,"losses_VAE":losses_VAE,"losses_X":losses_X,"losses_Y":losses_Y,"losses_KL":losses_KL}))
        plt.suptitle("Losses Analysis", y=1.02)
        plt.show()
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.lineplot(pd.DataFrame({"losses":losses,"losses_VAE":losses_VAE,"losses_X":losses_X,"losses_Y":losses_Y,"losses_KL":losses_KL}).iloc[m:,:])
        plt.suptitle("Losses Analysis-Focus", y=1.02)
        plt.show()
        print("losses_min:",np.min(losses))
        print("RMSE_X:",RMSE_X)
        print("RMSE_y",RMSE_y)
        print('wRMSE_y',wRMSE_y)
        print("MSE_X:",RMSE_X**2)
    if  plot_Model:
        sns.pairplot(encoded,diag_kind='kde')
        plt.show()
    # if plot_Model:
    #     sns.set(rc={'figure.figsize':(11.7,8.27)})
    #     sns.heatmap(np.round(pd.DataFrame(encoded.corr(method="spearman")),1),annot=True)
    #     sns.heatmap(np.round(pd.DataFrame(encoded.corr(method="pearson")),1),annot=True)
    #     plt.show()
    # if MC_ref is not None:
    #     MC_dif = (np.abs(MC_ref - np.abs(pd.DataFrame(encoded.corr(method="spearman")))).sum().sum())/2
    #     MC_filtre = MC_ref - np.eye(MC_ref.shape[1])
    #     MC_corr=np.abs(np.abs(pd.DataFrame(encoded.corr(method="spearman"))) * MC_filtre - MC_filtre).sum().sum()/2 
    if plot_Model:  
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.pairplot(decoded_x, diag_kind='kde')
        sns.pairplot(reconstruct_x, diag_kind='kde', hue='Type')
        sns.pairplot(decoded_y, diag_kind='kde')
        sns.pairplot(reconstruct_y, diag_kind='kde', hue='Type')
        
    if MC_ref is not None:
        return {'RMSE_X':RMSE_X,'RMSE_y':RMSE_y,'wRMSE_y':wRMSE_y,'MSE_X':RMSE_X**2, 'MC_dif':MC_dif,'MC_corr':MC_corr,"KL_R":KL_R}
    else:
        return {'RMSE_X':RMSE_X,'RMSE_y':RMSE_y,'wRMSE_y':wRMSE_y, 'MSE_X':RMSE_X**2, "KL_R":KL_R}

## Generation

In [ ]:
def generationXy(res, X, y, w=None, N=None, seed=None, mode="VAE", hmult=0.1):
    n = X.shape[0]
    if N is None:N=n
    if w is None:w=np.repeat(1,n)/n
    if seed is None: seed = np.random.choice(n, size=N, replace=True)
    inputsX0=X.copy()
    inputsy0=y.copy()
    if res is not None:
        inputsX=transformer(X)
        inputsy=transformer(y)
        if mode == "VAE":
            inputsX = np.array(inputsX)[seed,:]
            inputsy = np.array(inputsy)[seed,:]
            decoded_x, decoded_y, encoded, mu, logvar = res['model'](torch.FloatTensor(inputsX).to(device),torch.FloatTensor(inputsy).to(device))
            outputs_x=pd.DataFrame(decoded_x.cpu().detach())
            outputs_y=pd.DataFrame(decoded_y.cpu().detach())
            outputs_x.columns=X.columns
            outputs_y.columns=y.columns
        elif mode in ["kVAE","kAE"]:
            mu=res['mu']
            kde = gaussian_kde(mu.cpu().detach().numpy().T, bw_method = "silverman")
            H = kde.factor**2 * kde.covariance * hmult
            sim = np.random.multivariate_normal(mean=np.repeat(0,mu.shape[1]),cov=H,size=N)
            encoded = mu[seed,:] + torch.FloatTensor(sim).to(device)
            decoded_x, decoded_y = res['model'].decode(encoded)
            outputs_x=pd.DataFrame(decoded_x.cpu().detach())
            outputs_y=pd.DataFrame(decoded_y.cpu().detach())
            outputs_x.columns=X.columns
            outputs_y.columns=y.columns
        outputs_x = inv_transformer(X,outputs_x)
        outputs_y = inv_transformer(y,outputs_y)
    else:
        if mode == "kPCA":
            Xytrain=pd.concat([y,X],axis=1)
            pca = PCA(n_components=Xytrain.shape[1])
            scaler = StandardScaler()
            Xytrain_sc = scaler.fit_transform(Xytrain)
            Xyfact = pca.fit_transform(Xytrain_sc)
            inputs = Xyfact[graine,:]
            kde = gaussian_kde(Xyfact.T, bw_method = "silverman")
            H = kde.factor**2 * kde.covariance * hmult
            sim = np.random.multivariate_normal(mean=np.repeat(0,inputs.shape[1]),cov=H,size=N)
            synth = pd.DataFrame(inputs + sim)
            synth = pca.inverse_transform(synth)
            synth = pd.DataFrame(scaler.inverse_transform(synth))
            synth.columns=Xytrain.columns
            synth.set_index(Xytrain.iloc[graine,:].index,inplace=True)
            outputs_x = synth[X.columns]
            outputs_y = synth[y.columns]
        elif mode == "kKPCA":
            Xytrain=pd.concat([y,X],axis=1)
            pca = KernelPCA(n_components=Xytrain.shape[1], kernel='linear', fit_inverse_transform=True)
            scaler = StandardScaler()
            Xytrain_sc = scaler.fit_transform(Xytrain)
            Xyfact = pca.fit_transform(Xytrain_sc)
            inputs = Xyfact[graine,:]
            kde = gaussian_kde(Xyfact.T, bw_method = "silverman")
            H = kde.factor**2 * kde.covariance * hmult
            sim = np.random.multivariate_normal(mean=np.repeat(0,inputs.shape[1]),cov=H,size=N)
            synth = pd.DataFrame(inputs + sim)
            synth = pca.inverse_transform(synth)
            synth = pd.DataFrame(scaler.inverse_transform(synth))
            synth.columns=Xytrain.columns
            synth.set_index(Xytrain.iloc[graine,:].index,inplace=True)
            outputs_x = synth[X.columns]
            outputs_y = synth[y.columns]
        else:
            inputs0=pd.concat([inputsX0,inputsy0],axis=1).to_numpy()
            inputs=pd.concat([inputsX0.iloc[seed,:],inputsy0.iloc[seed,:]],axis=1).to_numpy()
            kde = gaussian_kde(inputs0.T, bw_method = "silverman")
            H = kde.factor**2 * kde.covariance * hmult
            sim = np.random.multivariate_normal(mean=np.repeat(0,inputs.shape[1]),cov=H,size=N)
            outputs = pd.DataFrame(inputs + sim)
            outputs_x = outputs.iloc[:,:-1]
            outputs_y = pd.DataFrame(outputs.iloc[:,-1])
            outputs_x.columns=X.columns
            outputs_y.columns=y.columns
    outputs_x.index=X.iloc[graine,:].index
    outputs_y.index=y.iloc[graine,:].index
    # outputs_x['seed']=seed
    return outputs_x,outputs_y

# ILLUSTRATION

## Training

In [ ]:
epoch_simu = 2000
plotModel=False

### Vanilla Xy

##### AE

In [ ]:
res_PCA_NL = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, betaKLD=0, epochs=epoch_simu, type_model="AE")
res_AE = res_PCA_NL

In [ ]:
res = PCA_NL_Analyze_y(res_PCA_NL, X_train, y_train, plot_Model=plotModel,w_Y=w_Y)

In [ ]:
res

##### $0-$VAE

In [ ]:
res_PCA_NL = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu, betaKLD=0,type_model="VAE")
res_0VAE = res_PCA_NL

In [ ]:
res = PCA_NL_Analyze_y(res_PCA_NL, X_train,y_train, plot_Model=plotModel,w_Y=w_Y)

In [ ]:
res

In [ ]:
if plotModel:sns.pairplot(pd.DataFrame(res_PCA_NL['mu'].cpu().detach().numpy()))

##### VAE

In [ ]:
# res_PCA_NL = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu, betaKLD=1,type_model="VAE", betaY=1)
# res_VAE=res_PCA_NL

In [ ]:
# res = PCA_NL_Analyze_y(res_PCA_NL, X_train,y_train, plot_Model=plotModel)

In [ ]:
# if plotModel:sns.pairplot(pd.DataFrame(res_PCA_NL['mu'].cpu().detach().numpy()))

##### $\beta-$VAE

In [ ]:
X_train.shape

In [ ]:
res_PCA_NL = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu,type_model="VAE")
res_BVAE = res_PCA_NL

In [ ]:
res_BVAE['encoded'].shape

In [ ]:
res = PCA_NL_Analyze_y(res_PCA_NL, X_train,y_train, plot_Model=plotModel,w_Y=w_Y)

In [ ]:
res

In [ ]:
if plotModel:sns.pairplot(pd.DataFrame(res_PCA_NL['mu'].cpu().detach().numpy()))

##### $\beta-$VAE - wMSE

In [ ]:
res_PCA_NL = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu,type_model="VAE",w_Y=w_Y)
res_BVAEw = res_PCA_NL

In [ ]:
res = PCA_NL_Analyze_y(res_PCA_NL, X_train,y_train, plot_Model=plotModel,w_Y=w_Y)

In [ ]:
res

In [ ]:
if plotModel:sns.pairplot(pd.DataFrame(res_PCA_NL['mu'].cpu().detach().numpy()))

In [ ]:
j=0
for i in range(X_train.shape[1]):
    sns.scatterplot(x=X_train.iloc[:,i],y=res_PCA_NL['mu'].cpu().detach()[:,j])

## Generation-Xy

In [ ]:
plotModel=False

In [ ]:
N = X_train.shape[0]

In [ ]:
graine = np.random.choice(X_train.shape[0], size=N, p=w_Y , replace=True)

In [ ]:
hmult=0

In [ ]:
y_grain = y_train.iloc[graine,:]

In [ ]:
sns.set(rc={'figure.figsize':(5,5)})
sns.histplot(y_grain, kde=True)
plt.show()
train_df = pd.DataFrame(y_train)
train_df['sample']='train'
tir_df = pd.DataFrame(y_grain)
tir_df['sample']='Tirage'
sns.histplot(data=pd.concat([train_df,tir_df],axis=0),x=lab_Y,hue='sample',kde=True, stat="density")

### Train

In [ ]:
synth_X, synth_y = generationXy(None,X_train, y_train, seed=graine, mode="kVAE",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_Train=synth

In [ ]:
np.abs((synth_Train-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### AE

In [ ]:
synth_X, synth_y = generationXy(res_AE,X_train, y_train, seed=graine, mode="kAE",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_kAE=synth

In [ ]:
np.abs((synth_kAE-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### 0-VAE

In [ ]:
synth_X, synth_y = generationXy(res_0VAE,X_train, y_train, seed=graine, mode="VAE",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_0VAE=synth

In [ ]:
np.abs((synth_0VAE-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### VAE

In [ ]:
# synth_X, synth_y = generationXy(res_VAE,X_train, y_train, seed=graine, mode="VAE",N=N, hmult=hmult)
# synth = pd.concat([synth_y,synth_X]],axis=1)
# synth_VAE=synth

In [ ]:
# if plotModel:sns.pairplot(synth)
# sns.histplot(synth[lab_Y])

### $\beta$-VAE

In [ ]:
synth_X, synth_y = generationXy(res_BVAE,X_train, y_train, seed=graine, mode="VAE",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_BVAE=synth

In [ ]:
np.abs((synth_BVAE-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### k$\beta$-VAE

In [ ]:
synth_X, synth_y = generationXy(res_BVAE,X_train, y_train, seed=graine, mode="kVAE",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_kBVAE=synth

In [ ]:
np.abs((synth_kBVAE-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### k$\beta$-VAE-wMSE

In [ ]:
synth_X, synth_y = generationXy(res_BVAEw,X_train, y_train, seed=graine, mode="kVAE",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_kBVAEw=synth

In [ ]:
np.abs((synth_kBVAEw-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### PCA

In [ ]:
synth_X, synth_y = generationXy(None,X_train, y_train, seed=graine, mode="kPCA",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_kPCA=synth

In [ ]:
np.abs((synth_kPCA-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

### KernelPCA

In [ ]:
Xytrain=pd.concat([y_train,X_train],axis=1)
pca = KernelPCA(n_components=Xytrain.shape[1], kernel='linear', fit_inverse_transform=True)
scaler = StandardScaler()
Xytrain_sc = scaler.fit_transform(Xytrain)
Xyfact = pca.fit_transform(Xytrain_sc)
inputs = Xyfact[graine,:]
kde = gaussian_kde(Xyfact.T, bw_method = "silverman")
H = kde.factor**2 * kde.covariance * hmult
sim = np.random.multivariate_normal(mean=np.repeat(0,inputs.shape[1]),cov=H,size=N)
synth = pd.DataFrame(inputs + sim)
synth = pca.inverse_transform(synth)
synth = pd.DataFrame(scaler.inverse_transform(synth))
synth.columns=Xytrain.columns
synth.set_index(Xytrain.iloc[graine,:].index,inplace=True)
outputs_x = synth[X_train.columns]
outputs_y = synth[y_train.columns]

In [ ]:
synth_X, synth_y = generationXy(None,X_train, y_train, seed=graine, mode="kKPCA",N=N, hmult=hmult)
synth = pd.concat([synth_y,synth_X],axis=1)
synth_kKPCA=synth

In [ ]:
np.abs((synth_kKPCA-Xy_train.iloc[graine,:])).sum()

In [ ]:
if plotModel:sns.pairplot(synth)
sns.histplot(synth[lab_Y])

## Prediction

### Initialisation

In [ ]:
h2o.init()

In [ ]:
test_data = h2o.H2OFrame(X_test)
X_col = X_test.columns
y_col = X_test.columns[colY]
X_col = X_col.drop(y_col).tolist()

In [ ]:
test_data

In [ ]:
def TirMix(train, synth, graine):
    train_graine = train.iloc[graine,:]
    doub=(train_graine.duplicated()*1).to_numpy()
    synth_mix = synth.copy()
    for i in range(synth_mix.shape[0]):
        if doub[i]==0:
            synth_mix.iloc[i,:]=train_graine.iloc[i,:]
    return np.round(synth_mix,10)

In [ ]:
def pred(train, test_data, Mix=False,Xy_train=None,graine=None):
    if Mix:
        print("mode TirMix")
        train_hat = TirMix(Xy_train, train, graine)
    else:
        train_hat=train.copy()
    train_hat = np.round(train_hat,10)
    w_test = IR_weighting(X_test[lab_Y])
    train_hat = h2o.H2OFrame(train_hat)
    aml = H2OAutoML(max_models=10, seed=42)
    print("train_hat.shape",train_hat.shape )
    aml.train(x=X_col, y=y_col, training_frame=train_hat)
    res_aml = aml.leaderboard
    preds = aml.predict(test_data)
    mse = mean_squared_error(preds.as_data_frame(),test_data[y_col].as_data_frame())
    wmse=(w_test*(preds.as_data_frame().to_numpy()-test_data[y_col].as_data_frame().to_numpy())**2).mean()
    mape = mean_absolute_percentage_error(preds.as_data_frame(),test_data[y_col].as_data_frame())
    mae = median_absolute_error(preds.as_data_frame(),test_data[y_col].as_data_frame())
    # egm = np.prod(np.abs(preds.as_data_frame().to_numpy()-test_data[y_col].as_data_frame().to_numpy()))**(1/test_data.shape[0])
    return {"res_aml" : res_aml, "preds" : preds, "mse" : mse, 'mape':mape, 'mae':mae, 'wmse':wmse}

### Mix=True

In [ ]:
Mix=True

#### Baseline: Train

In [ ]:
resPred_baseline = pred(Xy_train, test_data, Mix=False)
print(resPred_baseline['mse'])
print(resPred_baseline['wmse'])

#### Oversampling

In [ ]:
resPred_baseline = pred(Xy_train.iloc[graine,:], test_data, Mix, Xy_train, graine)
print(resPred_baseline['mse'])
print(resPred_baseline['wmse'])

#### SB_Train

In [ ]:
(synth_Train-Xy_train.iloc[graine,:]).sum()

In [ ]:
resPred_Train = pred(synth_Train, test_data, Mix, Xy_train, graine)
print(resPred_Train['mse'])
print(resPred_Train['wmse'])

#### PCA

In [ ]:
(synth_kPCA-Xy_train.iloc[graine,:]).sum()

In [ ]:
resPred_kPCA = pred(synth_kPCA, test_data, Mix, Xy_train, graine)
print(resPred_kPCA['mse'])
print(resPred_kPCA['wmse'])

#### KernelPCA

In [ ]:
resPred_kKPCA = pred(synth_kKPCA, test_data, Mix, Xy_train, graine)
print(resPred_kKPCA['mse'])
print(resPred_kKPCA['wmse'])

#### AE

In [ ]:
resPred_kAE = pred(synth_kAE, test_data, Mix, Xy_train, graine)
print(resPred_kAE['mse'])
print(resPred_kAE['wmse'])

#### 0VAE

In [ ]:
resPred_0VAE = pred(synth_0VAE, test_data, Mix, Xy_train, graine)
print(resPred_0VAE['mse'])
print(resPred_0VAE['wmse'])

#### BVAE

In [ ]:
resPred_BVAE = pred(synth_BVAE, test_data, Mix, Xy_train, graine)
print(resPred_BVAE['mse'])
print(resPred_BVAE['wmse'])

#### kBVAE

In [ ]:
resPred_kBVAE = pred(synth_kBVAE, test_data, Mix, Xy_train, graine)
print(resPred_kBVAE['mse'])
print(resPred_kBVAE['wmse'])

#### kBVAE-wMSE

In [ ]:
resPred_kBVAEw = pred(synth_kBVAEw, test_data, Mix, Xy_train, graine)
print(resPred_kBVAEw['mse'])
print(resPred_kBVAEw['wmse'])

In [ ]:
stop

## Boucle runs

In [ ]:
n_simus=10
test_size=0.4
train_size=0.6
n = X_train.shape[0]
N = n
Mix=True
hmult=0.1
alpha=1
epoch_simu=2000
benchmark = True
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(Xy_train)

In [ ]:
res_MSE_l =[]
res_MAPE_l=[]
res_MAE_l =[]
res_wMSE_l=[]
seed_samples=[860, 738, 945, 289, 473, 354, 945, 19, 30, 611] # seeds to get the same results than paper
np.random.seed()
for simu in range(n_simus):
        
    print("============================     simu : ",simu,"          ======================")
    
    res_MSE  =  []
    res_wMSE =  []
    res_MAE  =  []
    res_MAPE =  []
    
    # Train-test sampling
    w_Y=IR_weighting(data[lab_Y], alpha=alpha)
    # seed_sample = np.random.randint(1000)
    seed_sample =seed_samples[simu]
    np.random.seed(seed_sample)
    # seed_samples.append(seed_sample)
    split = trainTest(data,test_size,seed_sample,w=w_Y,train_size=train_size)
    X_train = split['X_train']
    Xy_train = X_train.copy()
    X_test = split['X_test']
    y_train =  X_train[[lab_Y]]
    X_train.drop(data.columns[colY], axis=1, inplace=True)
    w_Y=IR_weighting(y_train[lab_Y])
    test_data = h2o.H2OFrame(X_test)
    X_col = X_test.columns
    y_col = X_test.columns[colY]
    X_col = X_col.drop(y_col).tolist()
    
    
    # Model training
    res_AE = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, betaKLD=0, epochs=epoch_simu, type_model="AE")
    res_0VAE = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu, betaKLD=0,type_model="VAE")
    res_BVAE = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu,type_model="VAE")
    res_BVAEw = train_VAEy(X_train=X_train,y_train=y_train, seed=seed_sample, stacked=None, epochs=epoch_simu,type_model="VAE",w_Y=w_Y)
    
    
    # Generation
    ## DAVID
    graine = np.random.choice(n, size=N, p=w_Y , replace=True)
    synth_X, synth_y = generationXy(None,X_train, y_train, seed=graine, mode="kVAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_Train=synth
    synth_X, synth_y = generationXy(res_AE,X_train, y_train, seed=graine, mode="kAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_kAE=synth
    synth_X, synth_y = generationXy(res_0VAE,X_train, y_train, seed=graine, mode="VAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_0VAE=synth
    synth_X, synth_y = generationXy(res_BVAE,X_train, y_train, seed=graine, mode="VAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_BVAE=synth
    synth_X, synth_y = generationXy(res_BVAE,X_train, y_train, seed=graine, mode="kVAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_kBVAE=synth
    synth_X, synth_y = generationXy(res_BVAEw,X_train, y_train, seed=graine, mode="VAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_BVAEw=synth
    synth_X, synth_y = generationXy(res_BVAEw,X_train, y_train, seed=graine, mode="kVAE",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_kBVAEw=synth
    synth_X, synth_y = generationXy(None,X_train, y_train, seed=graine, mode="kPCA",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_kPCA=synth
    synth_X, synth_y = generationXy(None,X_train, y_train, seed=graine, mode="kKPCA",N=N, hmult=hmult)
    synth = pd.concat([synth_y,synth_X],axis=1)
    synth_kKPCA=synth
    
    # Benchmark
    if benchmark:
        ## TVAE
        warnings.simplefilter("ignore")
        synthesizer = TVAESynthesizer(metadata, epochs=epoch_simu)
        synthesizer.fit(Xy_train)
        synth_TVAE = synthesizer.sample(num_rows=N)
        ## CTGAN
        synthesizer = CTGANSynthesizer(metadata,epochs=epoch_simu)
        synthesizer.fit(Xy_train)
        synth_CTGAN = synthesizer.sample(num_rows=N)
        ## CopulaGAN
        synthesizer = CopulaGANSynthesizer(metadata,epochs=epoch_simu)
        synthesizer.fit(Xy_train)
        synth_CopGAN = synthesizer.sample(num_rows=N)
        ## ILR
        synth_ILRro = ro(data = Xy_train.reset_index().drop(columns="index", axis=1), y = lab_Y)#.iloc[:N,:]
        # synth_ILRro.set_index(Xy_train.index)
        synth_ILRsmote = smote(data = Xy_train.reset_index().drop(columns="index", axis=1), y = lab_Y,k=3)#.iloc[:N,:]
        # synth_ILRsmote.set_index(Xy_train.index)
        synth_ILRgn = gn(data = Xy_train.reset_index().drop(columns="index", axis=1), y = lab_Y, pert=hmult)#.iloc[:N,:]
        # synth_ILRgn.set_index(Xy_train.index)
        # try:
        #     synth_ILRadasyn = adasyn(data = Xy_train.reset_index().drop(columns="index", axis=1), y = lab_Y)#.iloc[:N,:]
        #     # synth_ILRadasyn.set_index(Xy_train.index)
        # except Exception:
        #     pass
    
    
    # Prediction
    resPred_baseline = pred(Xy_train, test_data, False, Xy_train, graine)
    resPred_OS = pred(Xy_train, test_data, Mix, Xy_train, graine)
    resPred_Train = pred(synth_Train, test_data, Mix, Xy_train, graine)
    resPred_kAE = pred(synth_kAE, test_data, Mix, Xy_train, graine)
    resPred_0VAE = pred(synth_0VAE, test_data, Mix, Xy_train, graine)
    resPred_BVAE = pred(synth_BVAE, test_data, Mix, Xy_train, graine)
    resPred_kBVAE = pred(synth_kBVAE, test_data, Mix, Xy_train, graine)
    resPred_BVAEw = pred(synth_BVAEw, test_data, Mix, Xy_train, graine)
    resPred_kBVAEw = pred(synth_kBVAEw, test_data, Mix, Xy_train, graine)
    resPred_kPCA = pred(synth_kPCA, test_data, Mix, Xy_train, graine)
    resPred_kKPCA = pred(synth_kKPCA, test_data, Mix, Xy_train, graine)
    if benchmark:
        resPred_TVAE = pred(synth_TVAE, test_data, Mix, Xy_train, graine)
        resPred_CTGAN = pred(synth_CTGAN, test_data, Mix, Xy_train, graine)
        resPred_CopGAN = pred(synth_CopGAN, test_data, Mix, Xy_train, graine)
        resPred_ILRro = pred(synth_ILRro, test_data, Mix=False, Xy_train=Xy_train, graine=graine)
        resPred_ILRsmote = pred(synth_ILRsmote, test_data, Mix=False, Xy_train=Xy_train, graine=graine)
        resPred_ILRgn = pred(synth_ILRgn, test_data, Mix=False, Xy_train=Xy_train, graine=graine)
        # resPred_ILRadasyn = pred(synth_ILRadasyn, test_data, Mix=False, Xy_train=Xy_train, graine=graine)
    warnings.resetwarnings()
    # Saving
    # if (resPred_kBVAEw['mse'] == min(resPred_kBVAEw['mse'],resPred_baseline['mse'],resPred_OS['mse'],
    #                                  resPred_Train['mse'],resPred_kAE['mse'],resPred_0VAE['mse'],resPred_BVAE['mse'],
    #                                  resPred_kBVAE['mse'],resPred_kPCA['mse'],resPred_kKPCA['mse'])):#,
    #                                  # resPred_TVAE['mse'], resPred_CTGAN['mse'],resPred_CopGAN['mse'],resPred_ILRro['mse'],resPred_ILRsmote['mse'],resPred_ILRgn['mse'])):
    #     seed_samples.append(seed_sample)
    res_MSE  =  pd.DataFrame({'train':"Baseline",'run_'+str(simu):[resPred_baseline['mse']]})
    res_MAE  =  pd.DataFrame({'train':"Baseline",'run_'+str(simu):[resPred_baseline['mae']]})
    res_MAPE =  pd.DataFrame({'train':"Baseline",'run_'+str(simu):[resPred_baseline['mape']]})
    res_wMSE =  pd.DataFrame({'train':"Baseline",'run_'+str(simu):[resPred_baseline['wmse']]})    

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"OS",'run_'+str(simu):[resPred_OS['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"OS",'run_'+str(simu):[resPred_OS['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"OS",'run_'+str(simu):[resPred_OS['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"OS",'run_'+str(simu):[resPred_OS['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"kTrain",'run_'+str(simu):[resPred_Train['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"kTrain",'run_'+str(simu):[resPred_Train['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"kTrain",'run_'+str(simu):[resPred_Train['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"kTrain",'run_'+str(simu):[resPred_Train['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"kAE",'run_'+str(simu):[resPred_kAE['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"kAE",'run_'+str(simu):[resPred_kAE['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"kAE",'run_'+str(simu):[resPred_kAE['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"kAE",'run_'+str(simu):[resPred_kAE['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"OVAE",'run_'+str(simu):[resPred_0VAE['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"OVAE",'run_'+str(simu):[resPred_0VAE['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"OVAE",'run_'+str(simu):[resPred_0VAE['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"OVAE",'run_'+str(simu):[resPred_0VAE['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"BVAE",'run_'+str(simu):[resPred_BVAE['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"BVAE",'run_'+str(simu):[resPred_BVAE['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"BVAE",'run_'+str(simu):[resPred_BVAE['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"BVAE",'run_'+str(simu):[resPred_BVAE['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"kBVAE",'run_'+str(simu):[resPred_kBVAE['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"kBVAE",'run_'+str(simu):[resPred_kBVAE['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"kBVAE",'run_'+str(simu):[resPred_kBVAE['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"kBVAE",'run_'+str(simu):[resPred_kBVAE['wmse']]})])
    
    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"BVAEw",'run_'+str(simu):[resPred_BVAEw['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"BVAEw",'run_'+str(simu):[resPred_BVAEw['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"BVAEw",'run_'+str(simu):[resPred_BVAEw['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"BVAEw",'run_'+str(simu):[resPred_BVAEw['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"kBVAEw",'run_'+str(simu):[resPred_kBVAEw['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"kBVAEw",'run_'+str(simu):[resPred_kBVAEw['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"kBVAEw",'run_'+str(simu):[resPred_kBVAEw['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"kBVAEw",'run_'+str(simu):[resPred_kBVAEw['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"kPCA",'run_'+str(simu):[resPred_kPCA['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"kPCA",'run_'+str(simu):[resPred_kPCA['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"kPCA",'run_'+str(simu):[resPred_kPCA['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"kPCA",'run_'+str(simu):[resPred_kPCA['wmse']]})])

    res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"kKPCA",'run_'+str(simu):[resPred_kKPCA['mse']]})])
    res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"kKPCA",'run_'+str(simu):[resPred_kKPCA['mae']]})])
    res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"kKPCA",'run_'+str(simu):[resPred_kKPCA['mape']]})])
    res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"kKPCA",'run_'+str(simu):[resPred_kKPCA['wmse']]})])

    if benchmark:
        res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"TVAE",'run_'+str(simu):[resPred_TVAE['mse']]})])
        res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"TVAE",'run_'+str(simu):[resPred_TVAE['mae']]})])
        res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"TVAE",'run_'+str(simu):[resPred_TVAE['mape']]})])
        res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"TVAE",'run_'+str(simu):[resPred_TVAE['wmse']]})])

        res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"CTGAN",'run_'+str(simu):[resPred_CTGAN['mse']]})])
        res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"CTGAN",'run_'+str(simu):[resPred_CTGAN['mae']]})])
        res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"CTGAN",'run_'+str(simu):[resPred_CTGAN['mape']]})])
        res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"CTGAN",'run_'+str(simu):[resPred_CTGAN['wmse']]})])

        res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"CopGAN",'run_'+str(simu):[resPred_CopGAN['mse']]})])
        res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"CopGAN",'run_'+str(simu):[resPred_CopGAN['mae']]})])
        res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"CopGAN",'run_'+str(simu):[resPred_CopGAN['mape']]})])
        res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"CopGAN",'run_'+str(simu):[resPred_CopGAN['wmse']]})])

        res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"ILRro",'run_'+str(simu):[resPred_ILRro['mse']]})])
        res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"ILRro",'run_'+str(simu):[resPred_ILRro['mae']]})])
        res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"ILRro",'run_'+str(simu):[resPred_ILRro['mape']]})])
        res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"ILRro",'run_'+str(simu):[resPred_ILRro['wmse']]})])

        res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"ILRsmote",'run_'+str(simu):[resPred_ILRsmote['mse']]})])
        res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"ILRsmote",'run_'+str(simu):[resPred_ILRsmote['mae']]})])
        res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"ILRsmote",'run_'+str(simu):[resPred_ILRsmote['mape']]})])
        res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"ILRsmote",'run_'+str(simu):[resPred_ILRsmote['wmse']]})])

        res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"ILRgn",'run_'+str(simu):[resPred_ILRgn['mse']]})])
        res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"ILRgn",'run_'+str(simu):[resPred_ILRgn['mae']]})])
        res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"ILRgn",'run_'+str(simu):[resPred_ILRgn['mape']]})])
        res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"ILRgn",'run_'+str(simu):[resPred_ILRgn['wmse']]})])

        # res_MSE  =  pd.concat([res_MSE,pd.DataFrame({'train':"ILRadasyn",'run_'+str(simu):[resPred_ILRadasyn['mse']]})])
        # res_MAE  =  pd.concat([res_MAE,pd.DataFrame({'train':"ILRadasyn",'run_'+str(simu):[resPred_ILRadasyn['mae']]})])
        # res_MAPE =  pd.concat([res_MAPE,pd.DataFrame({'train':"ILRadasyn",'run_'+str(simu):[resPred_ILRadasyn['mape']]})])
        # res_wMSE =  pd.concat([res_wMSE,pd.DataFrame({'train':"ILRadasyn",'run_'+str(simu):[resPred_ILRadasyn['wmse']]})])


    if len(res_MSE_l)==0:
        res_MSE_l  = res_MSE
        res_MAE_l  =  res_MAE
        res_MAPE_l =  res_MAPE
        res_wMSE_l =  res_wMSE
    else:
        res_MSE_l  = res_MSE_l.merge(res_MSE)
        res_MAE_l  = res_MAE_l.merge(res_MAE)
        res_MAPE_l = res_MAPE_l.merge(res_MAPE)
        res_wMSE_l  = res_wMSE_l.merge(res_wMSE)

res_MSE_l.set_index('train', inplace=True)
res_MAE_l.set_index('train', inplace=True)
res_MAPE_l.set_index('train', inplace=True)
res_wMSE_l.set_index('train', inplace=True)


In [ ]:
sns.set(rc={'figure.figsize':(5,5)})
sns.heatmap(pd.DataFrame(res_MSE_l.mean(axis=1)), annot=True, cmap='RdYlGn_r',fmt='.2f')
plt.show()
sns.heatmap(pd.DataFrame(res_MSE_l.std(axis=1)), annot=True, cmap='RdYlGn_r',fmt='.2f')

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.boxplot(res_MSE_l.transpose(), orient='h')

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.boxplot(res_wMSE_l.transpose(), orient='h')

In [ ]:
res_wMSE_l.mean(axis=1)

In [ ]:
seed_samples

In [ ]:
res_MSE_l.rank(axis=0)

In [ ]:
res_MSE_l.to_csv(rep+"/res_MSE_l.csv")
res_MAE_l.to_csv(rep+"/res_MAE_l.csv")
res_MAPE_l.to_csv(rep+"/res_MAPE_l.csv")
res_wMSE_l.to_csv(rep+"/res_wMSE_l.csv")

In [ ]:
pd.DataFrame(seed_samples).to_csv(rep+"/seed_samples.csv", index=False)

In [ ]:
res_MSE_l.rank(axis=0)